# Data Cleaning
In this session, we will do same basic cleaning steps on the main datasets

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import shapefile as shp
import pandas as pd
import numpy as np
import os

In [3]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Data Cleaning")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "9g") 
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("OFF")

24/09/11 09:55:19 WARN Utils: Your hostname, LAPTOP-406UJ3L3 resolves to a loopback address: 127.0.1.1; using 172.22.101.212 instead (on interface eth0)
24/09/11 09:55:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/11 09:55:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/11 09:55:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Load dataset

We have 3 kinds of dataset: transactions, consumer, merchant. We aim to find top 100 merchants we should accept, so, let's look at merchant dataset.

## 1. Merchants

In [ ]:
# Information on merchants
merchant = spark.read.parquet("../data/tables/part_1/tbl_merchants.parquet")

# Information on merchant's fraud probability
merchant_fraud_prob = pd.read_csv("../data/tables/part_1/merchant_fraud_probability.csv")

In [ ]:
print(f"Number of rows: {merchant.count()}")
merchant.limit(5)

In [ ]:
merchant_fraud_prob.count()

We should look at the data type of each column.

In [ ]:
merchant.printSchema()

In [ ]:
merchant_fraud_prob.dtypes

### Comment
- The merchant dataset contains their names, tags (seems like stuff they sell), and the abn.
- For the probability dataset, they record the datetime of an order and the probability it is broken?.
- Two tables share the `merchant_abn` column.

In [ ]:
# Look at the tags column
merchant.select("tags").limit(5).collect()

In [ ]:
# Check duplicate
print(f"Number of duplicates: {merchant.select('merchant_abn').count() - merchant.select('merchant_abn').distinct().count()}")

In [ ]:
# Check null value
print(f"NaN in merchant detail: {merchant.filter(F.col('merchant_abn').isNull() | F.col('tags').isNull() | F.col('name').isNull()).count()}")
print(f"Nan in merchant fraud rate:\n{merchant_fraud_prob.isna().sum()}")

In [ ]:
# Number of merchant with a fraud rate
print(f"Number of merchant with a fraud rate: {len(merchant_fraud_prob.merchant_abn.unique())}")

In [ ]:
# The abn with fraud but not in  merchant info
abn_not_in_merchant = spark.createDataFrame(pd.DataFrame(merchant_fraud_prob.merchant_abn)).subtract(merchant.select('merchant_abn'))
abn_not_in_merchant.show()

In [ ]:
print(f"Number of abn not in merchant dataset: {abn_not_in_merchant.count()}")
merchant.filter(F.col("merchant_abn") == 82999039227).show()
merchant_fraud_prob[merchant_fraud_prob.merchant_abn == 82999039227]

In [ ]:
# Time range for fraud probability
merchant_fraud_prob.loc[:,"order_datetime"] = pd.to_datetime(merchant_fraud_prob.order_datetime)
merchant_fraud_prob.order_datetime.sort_values()

The time range recoded in the merchant fraud dataset is from 25/03/2021 to 27/02/2022.

In [ ]:
# Check if fraud prob is in a valid range
merchant_fraud_prob.fraud_probability.describe()

The fraud probability is in good range (from 0 to 1), with the highest probability at 94%, which was found to belong to an abn not existing in the merchant dataset. 

### Consideration
- We need to separate each tupe into 3 values: stuff_type, ?, take_rate
- How to deal with the text containing goods: remove stop words, dec2vec, count2vec, etc.
- Convert `order_datetime` to datetime data type.

## 2. Consumer

In [ ]:
# Information on consumer
consumer_user_detail = spark.read.parquet("../data/tables/part_1/consumer_user_details.parquet")
consumer = pd.read_csv("../data/tables/part_1/tbl_consumer.csv", delimiter="|")

# Information on consumer's fraud probability
consumer_fraud_prob = pd.read_csv("../data/tables/part_1/consumer_fraud_probability.csv")

In [ ]:
print(f"Shapes: {consumer.shape}")
consumer.head(5)

In [ ]:
print(f"Shape: {consumer_fraud_prob.shape}")
consumer_fraud_prob.head(5)

In [ ]:
print(f"Number of rows: {consumer_user_detail.count()}")
consumer_user_detail.limit(5)

### Comment
- The consumer dataset contains their name, address, state, postcode, gender and consumer_id. While the consumer user details dataset contain corresponding consumer_id for each user_id.
- Consumer fraud probability and consumer dataset can be merged through consumer user detail dataset.

In [ ]:
# Check duplicate in user_id and consumer_id
print(f"Number of duplicates in user_id: {consumer_user_detail.select('user_id').count() - consumer_user_detail.select('user_id').distinct().count()}")

print(f"Number of duplicates in consumer_id: {consumer_user_detail.select('consumer_id').count() - consumer_user_detail.select('consumer_id').distinct().count()}")

**Question**: Why do we need both consumer_id and user_id (related to database)?

An user id has to be in a range of 1 and 499,999, while consumer_id can be random?

In [ ]:
# Check null values
print(f"Nan in merchant fraud rate:\n{consumer_fraud_prob.isna().sum()}")
print(f"Nan in merchant fraud rate:\n{consumer.isna().sum()}")

In [ ]:
# Check if user id range in other dataframes is valid.
print(f"Max user id in the fraud record: {max(consumer_fraud_prob.user_id)}")
print(f"Min user id in the fraud record: {min(consumer_fraud_prob.user_id)}")

In [ ]:
# Check for the consumer id without a valid corresponding user id
invalid_consumer_id = spark.createDataFrame(pd.DataFrame(consumer.consumer_id)).subtract(consumer_user_detail.select('consumer_id'))
invalid_consumer_id.limit(1)

There are no problems in consumer_id and user_id.

In [ ]:
# Time range for fraud probability
consumer_fraud_prob.loc[:,"order_datetime"] = pd.to_datetime(consumer_fraud_prob.order_datetime)
consumer_fraud_prob.order_datetime.sort_values()

In [ ]:
# Check if fraud prob is in a valid range
consumer_fraud_prob.fraud_probability.describe()

It has a suitable time range and probability. Overall, we just need to convert order_datetime to datetime data type.

## 3. Transaction

This dataset contains details for each transaction between a merchant and a user.

In [11]:
# Read transaction dataset
transaction1 = spark.read.parquet("../data/tables/part_2/*")
transaction2 = spark.read.parquet("../data/tables/part_3/*")
transaction3 = spark.read.parquet("../data/tables/part_4/*")

In [12]:
# Combine datasets into a single DataFrame
transaction = transaction1.union(transaction2).union(transaction3)
transaction.count() # Number of rows

14195505

In [ ]:
transaction.printSchema()

In [ ]:
transaction.limit(5)

#### 1. Check for missing values

In [ ]:
# List of columns to check
cols = transaction.columns

for col_name in cols:
    # Filter rows where values are null and count them
    null_count = transaction.filter(F.col(col_name).isNull()).count()

    if null_count > 0:
        print(f"Number of rows with null {col_name}: {null_count}")

So there are no missing values in any columns.

In [ ]:
for col in ['user_id', 'dollar_value', 'order_datetime']:
    print(f"Max of {col}: {transaction.agg({col: 'max'})}")
    print(f"Min of {col}: {transaction.agg({col: 'min'})}")

### Comment
- Although we have 499,999 user ids, but only at most 24081 of them made a transaction within the provided time range.
- the `dollar_value` range seems strange as the minimum value is almost 0. We may need to do some outlier analysis for it.
- The time range of the transaction dataset is wider than the time range of other datasets, which can lead to missing data when we joined every table together.

# Create Cleaning Function

At this step, only the column `tags` of merchant dataset needs to be preprocessed. We decide to convert the tbl_merchants.parquet to csv because of its small size.

In [4]:
import sys
sys.path.append('../scripts')
from etl import clean_merchant_df

In [5]:
# Apply clean function
merchant = spark.read.parquet("../data/tables/part_1/tbl_merchants.parquet")
merchant.show()

+--------------------+--------------------+------------+
|                name|                tags|merchant_abn|
+--------------------+--------------------+------------+
|       Felis Limited|((furniture, home...| 10023283211|
|Arcu Ac Orci Corp...|([cable, satellit...| 10142254217|
|    Nunc Sed Company|([jewelry, watch,...| 10165489824|
|Ultricies Digniss...|([wAtch, clock, a...| 10187291046|
| Enim Condimentum PC|([music shops - m...| 10192359162|
|       Fusce Company|[(gift, card, nov...| 10206519221|
|Aliquam Enim Inco...|[(computers, comP...| 10255988167|
|    Ipsum Primis Ltd|[[watch, clock, a...| 10264435225|
|Pede Ultrices Ind...|([computer progra...| 10279061213|
|           Nunc Inc.|[(furniture, home...| 10323485998|
|Facilisis Facilis...|([computers, comp...| 10342410215|
|      Odio Institute|((equipment, tool...| 10346855916|
|    Rutrum Justo Ltd|([music shops - m...| 10364012396|
|   Tellus Foundation|[[artist supply a...| 10385011947|
|      Sed Et Company|([florist

In [6]:
clean_merchant_df(merchant)

In [8]:
cleaned_merchant_df = spark.read.parquet("../data/curated/part_1/tbl_merchants.parquet")

In [9]:
cleaned_merchant_df.show()

+--------------------+------------+--------------------+-------------+---------+
|                name|merchant_abn|               goods|revenue_level|take_rate|
+--------------------+------------+--------------------+-------------+---------+
|       Felis Limited| 10023283211|furniture, home f...|            e|     0.18|
|Arcu Ac Orci Corp...| 10142254217|cable, satellite,...|            b|     4.22|
|    Nunc Sed Company| 10165489824|jewelry, watch, c...|            b|      4.4|
|Ultricies Digniss...| 10187291046|watch, clock, and...|            b|     3.29|
| Enim Condimentum PC| 10192359162|music shops - mus...|            a|     6.33|
|       Fusce Company| 10206519221|gift, card, novel...|            a|     6.34|
|Aliquam Enim Inco...| 10255988167|computers, comput...|            b|     4.32|
|    Ipsum Primis Ltd| 10264435225|watch, clock, and...|            c|     2.39|
|Pede Ultrices Ind...| 10279061213|computer programm...|            a|     5.71|
|           Nunc Inc.| 10323

## Feature Engineering

In [13]:
aggregrated_trans_df = transaction.groupBy("merchant_abn").agg(F.sum("dollar_value"), F.count("dollar_value"))

In [14]:
aggregrated_trans_df.show()

+------------+------------------+-------------------+
|merchant_abn| sum(dollar_value)|count(dollar_value)|
+------------+------------------+-------------------+
| 38700038932| 9546185.360697314|               7132|
| 83412691377| 498536.5816973136|              14288|
| 15613631617| 543030.5313328261|               1785|
| 19839532017|          113982.0|                726|
| 73256306726| 1496967.159114219|               5263|
| 35344855546|134737.25046268434|               1522|
| 24406529929|  280125.230897664|               4184|
| 73841664453| 83314.29045077678|                959|
| 78916025936|21015.671086215083|                 67|
| 60654402457| 17030.97660860815|                199|
| 92202115241|39887.588144581016|                120|
| 52763133264|14217.064637334852|                125|
| 48214071373|162100.66187954217|                540|
| 41956465747|63674.266719753636|                279|
| 56395390867| 46747.77965910106|                 47|
| 34440496342|19425.35882870

Creating a scaled revenue as as feature, we take log of the ratio.

In [15]:
aggregrated_trans_df = aggregrated_trans_df.withColumn("log_ratio", F.log(F.col("sum(dollar_value)") / F.col("count(dollar_value)")))

In [16]:
aggregrated_trans_df.show()

+------------+------------------+-------------------+------------------+
|merchant_abn| sum(dollar_value)|count(dollar_value)|         log_ratio|
+------------+------------------+-------------------+------------------+
| 38700038932| 9546185.360697314|               7132| 7.199305215006568|
| 83412691377| 498536.5816973136|              14288| 3.552256946120572|
| 15613631617| 543030.5313328261|               1785| 5.717747130253923|
| 19839532017|          113982.0|                726| 5.056245805348308|
| 73256306726| 1496967.159114219|               5263|5.6504952400078965|
| 35344855546|134737.25046268434|               1522| 4.483301329640816|
| 24406529929|  280125.230897664|               4184| 4.203969029624668|
| 73841664453| 83314.29045077678|                959|  4.46448429258872|
| 78916025936|21015.671086215083|                 67| 5.748331061215046|
| 60654402457| 17030.97660860815|                199| 4.449484293653718|
| 92202115241|39887.588144581016|                12

In [17]:
cleaned_merchant_df = cleaned_merchant_df.join(aggregrated_trans_df, on="merchant_abn", how="left")

In [18]:
cleaned_merchant_df = cleaned_merchant_df.withColumn("unscaled_earning", (F.col("take_rate")/100 * F.col("sum(dollar_value)")))

In [19]:
cleaned_merchant_df.limit(5)

merchant_abn,name,goods,revenue_level,take_rate,sum(dollar_value),count(dollar_value),log_ratio,unscaled_earning
10023283211,Felis Limited,"furniture, home f...",e,0.18,703277.7114509277,3261,5.373717954622449,1265.8998806116697
10142254217,Arcu Ac Orci Corp...,"cable, satellite,...",b,4.22,118356.1460726035,3036,3.663157406405673,4994.6293642638675
10165489824,Nunc Sed Company,"jewelry, watch, c...",b,4.4,56180.473857030534,5,9.326886622809969,2471.9408497093436
10192359162,Enim Condimentum PC,music shops - mus...,a,6.33,177980.5054563818,385,6.136185969080728,11266.165995388967
10187291046,Ultricies Digniss...,"watch, clock, and...",b,3.29,39693.73038743403,336,4.7718373694908784,1305.9237297465795


In [20]:
cleaned_merchant_df.write.mode('overwrite').parquet('../data/curated/part_1/clean_merchant.parquet')

In [21]:
cleaned_merchant_df.count()

4026